In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_csv('car.csv')
data.head()

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,75721,ford,Ford Kuga,black,05/2023,2023,38490,140,190,Automatic,Hybrid,"5,4 l/100 km",124 g/km,100.0,ST-Line Hybrid Adapt.LED+Head-Up-Display Klima
1,80184,hyundai,Hyundai i10,black,09/2018,2018,11555,49,67,Manual,Petrol,"4,6 l/100 km",106 g/km,27782.0,"blue Trend,Viele Extras,Top-Zustand"
2,19864,audi,Audi Q4 e-tron,grey,05/2021,2021,48886,125,170,Automatic,Electric,NaN,0 g/km,4247.0,35 e-tron S line/Matrix/Pano/ACC/SONOS/LM 21
3,76699,honda,Honda CR-V,red,07/2018,2018,24490,114,155,Automatic,Petrol,"7,5 l/100 km",175 g/km,57000.0,2.0 Lifestyle Plus Automatik Navi FAP
4,92991,kia,Kia Sportage,black,02/2023,2023,34990,110,150,Manual,Petrol,"5,9 l/100 km",150 g/km,7500.0,"1.6 T 48V 2WD Spirit LED, WR"


In [ ]:
data.describe(include='all')

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
count,100000.000000,100000,100000,99924,99999,100000,100000,99925,99928,100000,100000,88761,100000,9.993200e+04,100000
unique,NaN,29,590,14,369,49,11652,490,480,4,54,404,937,NaN,79777
top,NaN,audi,Ford Focus,black,05/2023,2019,19990,110,150,Automatic,Petrol,"5,1 l/100 km",- (g/km),NaN,Titanium
freq,NaN,21161,3875,24124,1846,12056,665,8521,8521,53262,54290,3257,15811,NaN,174
mean,49999.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.559891e+04,NaN
std,28867.657797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.929713e+04,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN
25%,24999.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.550000e+04,NaN
50%,49999.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.800000e+04,NaN
75%,74999.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.270000e+05,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                100000 non-null  int64  
 1   brand                     100000 non-null  object 
 2   model                     100000 non-null  object 
 3   color                     99924 non-null   object 
 4   registration_date         99999 non-null   object 
 5   year                      100000 non-null  object 
 6   price_in_euro             100000 non-null  object 
 7   power_kw                  99925 non-null   object 
 8   power_ps                  99928 non-null   object 
 9   transmission_type         100000 non-null  object 
 10  fuel_type                 100000 non-null  object 
 11  fuel_consumption_l_100km  88761 non-null   object 
 12  fuel_consumption_g_km     100000 non-null  object 
 13  mileage_in_km             99932 non-null   fl

# What the model learns from:
- Technical specs: power_kw, fuel_consumption, mileage_in_km
- Age factors: car_age, registration_year  
- Economic factors: price_in_euro
- Performance metrics: power_ps, fuel_consumption_g_km

# Problem with keeping 'model' column:
- **Data Leakage**: The model name often directly encodes the price
  (e.g., "BMW 7 Series" vs "Toyota Yaris" - you don't need ML to know which is more expensive)

- **High Cardinality**: Too many unique values (hundreds of car models)
- **Overfitting**: Model would memorize specific car models rather than learning general patterns
- **Poor Generalization**: Wouldn't work well for new/unseen car models

# Without dropping:
- R² = 0.95 (seems great!)
- But fails completely on new car models
- Model just memorized: "BMW 7 Series" = €80,000, "Toyota Yaris" = €15,000

# After dropping:
- R² = 0.82 (still good)
- Actually understands: higher power + lower mileage + newer = more expensive
- Works on any car model

In [ ]:
# In Model column There are three hundred and twelve unique models.
# That's something really hard to implement and a regression that would mean more than 300 dummies,
data = data.drop(labels='model', axis=1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                100000 non-null  int64  
 1   brand                     100000 non-null  object 
 2   color                     99924 non-null   object 
 3   registration_date         99999 non-null   object 
 4   year                      100000 non-null  object 
 5   price_in_euro             100000 non-null  object 
 6   power_kw                  99925 non-null   object 
 7   power_ps                  99928 non-null   object 
 8   transmission_type         100000 non-null  object 
 9   fuel_type                 100000 non-null  object 
 10  fuel_consumption_l_100km  88761 non-null   object 
 11  fuel_consumption_g_km     100000 non-null  object 
 12  mileage_in_km             99932 non-null   float64
 13  offer_description         100000 non-null  ob

In [ ]:
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                100000 non-null  int64  
 1   brand                     100000 non-null  object 
 2   color                     99924 non-null   object 
 3   registration_date         99999 non-null   object 
 4   year                      100000 non-null  object 
 5   price_in_euro             100000 non-null  object 
 6   power_kw                  99925 non-null   object 
 7   power_ps                  99928 non-null   object 
 8   transmission_type         100000 non-null  object 
 9   fuel_type                 100000 non-null  object 
 10  fuel_consumption_l_100km  88761 non-null   object 
 11  fuel_consumption_g_km     100000 non-null  object 
 12  mileage_in_km             99932 non-null   float64
 13  offer_description         100000 non-null  ob

,0
Unnamed: 0,0
brand,0
color,76
registration_date,1
year,0
price_in_euro,0
power_kw,75
power_ps,72
transmission_type,0
fuel_type,0


# Problem with keeping 'description' column:
- **Free-text Field**: Contains unstructured, verbose descriptions
- **Feature Explosion**: Would require complex NLP processing (TF-IDF, embeddings)
- **Noise**: Contains subjective opinions, marketing language, irrelevant details
- **Computationally Expensive**: Would dramatically increase dimensionality

In [ ]:
# Drop obvious non-features
drop_cols = [c for c in ['Unnamed: 0', 'offer_description'] if c in data.columns]
data = data.drop(columns=drop_cols)

In [ ]:
# Fix missing or invalid 'registration_date' by replacing '#######' with NaT (Not a Time)
data['registration_date'] = pd.to_datetime(data['registration_date'], errors='coerce')

# Optionally, you can replace NaT values with the most common date (mode)
mode_registration_date = data['registration_date'].mode()[0]
data['registration_date'].fillna(mode_registration_date, inplace=True)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   brand                     100000 non-null  object        
 1   color                     99924 non-null   object        
 2   registration_date         100000 non-null  datetime64[ns]
 3   year                      100000 non-null  object        
 4   price_in_euro             100000 non-null  object        
 5   power_kw                  99925 non-null   object        
 6   power_ps                  99928 non-null   object        
 7   transmission_type         100000 non-null  object        
 8   fuel_type                 100000 non-null  object        
 9   fuel_consumption_l_100km  88761 non-null   object        
 10  fuel_consumption_g_km     100000 non-null  object        
 11  mileage_in_km             99932 non-null   float64       
dtypes: 

In [ ]:
import re

# --- Helper: pull a numeric value out of a string (handles "5,9 l/100 km", "124 g/km", "€34,990", etc.)
def extract_number(s):
    if pd.isna(s):
        return np.nan
    # grab first number (allows comma or dot as decimal)
    m = re.search(r'[-+]?\d+(?:[.,]\d+)?', str(s))
    if not m:
        return np.nan
    return float(m.group(0).replace(',', '.'))

# Assuming 'data' is already loaded
# Handle numeric conversions for columns

# Convert 'price_in_euro' (remove € if exists, and handle commas)
data['price_in_euro'] = data['price_in_euro'].apply(extract_number)

# Convert 'fuel_consumption_l_100km' (remove ' l/100 km' and handle commas)
data['fuel_consumption_l_100km'] = data['fuel_consumption_l_100km'].astype(str)  # Ensure it's a string
data['fuel_consumption_l_100km'] = data['fuel_consumption_l_100km'].apply(lambda x: x.replace(' l/100 km', '').replace(',', '.'))
data['fuel_consumption_l_100km'] = pd.to_numeric(data['fuel_consumption_l_100km'], errors='coerce')

# Convert 'fuel_consumption_g_km' (remove ' g/km' and handle commas)
data['fuel_consumption_g_km'] = data['fuel_consumption_g_km'].apply(extract_number)

# Convert 'power_kw' and 'power_ps' to numeric
data['power_kw'] = pd.to_numeric(data['power_kw'], errors='coerce')
data['power_ps'] = pd.to_numeric(data['power_ps'], errors='coerce')

# Convert 'mileage_in_km' to numeric (already float64)
data['mileage_in_km'] = pd.to_numeric(data['mileage_in_km'], errors='coerce')

# Convert 'year' to numeric (if it's not already)
data['year'] = pd.to_numeric(data['year'], errors='coerce')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   brand                     100000 non-null  object        
 1   color                     99924 non-null   object        
 2   registration_date         100000 non-null  datetime64[ns]
 3   year                      99947 non-null   float64       
 4   price_in_euro             99960 non-null   float64       
 5   power_kw                  99884 non-null   float64       
 6   power_ps                  99907 non-null   float64       
 7   transmission_type         100000 non-null  object        
 8   fuel_type                 100000 non-null  object        
 9   fuel_consumption_l_100km  87982 non-null   float64       
 10  fuel_consumption_g_km     83884 non-null   float64       
 11  mileage_in_km             99932 non-null   float64       
dtypes: 

In [ ]:
data.isnull().sum()

,0
brand,0
color,76
registration_date,0
year,53
price_in_euro,40
power_kw,116
power_ps,93
transmission_type,0
fuel_type,0
fuel_consumption_l_100km,12018


In [ ]:
# Step 1: Drop rows where the target column 'price_in_euro' is missing
data.dropna(subset=['price_in_euro'], inplace=True)

# Step 2: Handle missing values in numeric columns (fill with median) excluding the target column 'price_in_euro'
numeric_columns = ['power_kw', 'power_ps', 'fuel_consumption_l_100km', 'fuel_consumption_g_km', 'mileage_in_km']
for col in numeric_columns:
    if col in data.columns:
        median_value = data[col].median()
        data[col].fillna(median_value, inplace=True)

# Step 3: Handle missing values in categorical columns (fill with mode) excluding the target column 'price_in_euro'
categorical_columns = ['color', 'registration_date', 'year', 'transmission_type', 'fuel_type', 'brand']
for col in categorical_columns:
    if col in data.columns:
        mode_value = data[col].mode()[0]  # mode()[0] gets the most frequent value
        data[col].fillna(mode_value, inplace=True)

# Step 4: Checking the result to ensure no missing values are left
print(data.isnull().sum())


brand                       0
color                       0
registration_date           0
year                        0
price_in_euro               0
power_kw                    0
power_ps                    0
transmission_type           0
fuel_type                   0
fuel_consumption_l_100km    0
fuel_consumption_g_km       0
mileage_in_km               0
dtype: int64


In [ ]:
# 5. Standardize categorical values to lowercase
data['fuel_type'] = data['fuel_type'].str.lower() # This line caused the error
data['transmission_type'] = data['transmission_type'].str.lower() # This line caused the error
data['color'] = data['color'].str.lower() # This line caused the error
data['brand'] = data['brand'].str.lower() # This line caused the error

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99960 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   brand                     99960 non-null  object        
 1   color                     99960 non-null  object        
 2   registration_date         99960 non-null  datetime64[ns]
 3   year                      99960 non-null  float64       
 4   price_in_euro             99960 non-null  float64       
 5   power_kw                  99960 non-null  float64       
 6   power_ps                  99960 non-null  float64       
 7   transmission_type         99960 non-null  object        
 8   fuel_type                 99960 non-null  object        
 9   fuel_consumption_l_100km  99960 non-null  float64       
 10  fuel_consumption_g_km     99960 non-null  float64       
 11  mileage_in_km             99960 non-null  float64       
dtypes: datetime64[ns](1), f

In [ ]:
numeric_cols = ['price_in_euro', 'power_kw', 'power_ps',
                'fuel_consumption_l_100km', 'fuel_consumption_g_km',
                'mileage_in_km']

# Remove outliers in 'price_in_euro' using the IQR method
Q1 = data['price_in_euro'].quantile(0.25)
Q3 = data['price_in_euro'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier thresholds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove rows that are outside the IQR range
data = data[(data['price_in_euro'] >= lower_bound) & (data['price_in_euro'] <= upper_bound)]


In [ ]:
# Feature engineering
data['registration_year'] = data['registration_date'].dt.year
data['registration_month'] = data['registration_date'].dt.month
current_year = pd.to_datetime('today').year
data['car_age'] = current_year - data['registration_year']

In [ ]:
# Use MinMaxScaler instead of StandardScaler to avoid negative values
features_to_standardize = ['power_kw', 'power_ps', 'fuel_consumption_l_100km', 'fuel_consumption_g_km', 'mileage_in_km']
scaler = MinMaxScaler()  # This scales to [0, 1] range - NO NEGATIVE VALUES
data_standardized = data[features_to_standardize].copy()
data_standardized[features_to_standardize] = scaler.fit_transform(data_standardized[features_to_standardize])
data[features_to_standardize] = data_standardized[features_to_standardize]

In [ ]:
# One-Hot Encoding with integers instead of booleans
data = pd.get_dummies(
    data,
    columns=['brand', 'color', 'transmission_type', 'fuel_type'],
    drop_first=True,
    dtype=int   # ensures 0/1 instead of True/False
)


In [ ]:
# Keep readable date format
data['registration_date'] = data['registration_date'].dt.strftime('%Y-%m-%d')

In [ ]:
print(data.head())

  registration_date    year  price_in_euro  power_kw  power_ps  \
0        2023-05-01  2023.0        38490.0  0.069154  0.233622   
1        2018-09-01  2018.0        11555.0  0.023881  0.081582   
2        2021-05-01  2021.0        48886.0  0.061692  0.208900   
3        2018-07-01  2018.0        24490.0  0.056219  0.190358   
4        2023-02-01  2023.0        34990.0  0.054229  0.184178   

   fuel_consumption_l_100km  fuel_consumption_g_km  mileage_in_km  \
0                  0.002669               0.061295       0.000026   
1                  0.002274               0.052397       0.007311   
2                  0.002867               0.000000       0.001118   
3                  0.003707               0.086505       0.015000   
4                  0.002916               0.074147       0.001974   

   registration_year  registration_month  ...  fuel_type_diesel hybrid  \
0               2023                   5  ...                        0   
1               2018                   9

In [ ]:
# Drop redundant columns like 'registration_year' if it’s already derived from 'registration_date'
data.drop(columns=['registration_year'], inplace=True, errors='ignore')  # Ignore if column doesn't exist


In [ ]:
# Save Preprocessed Dataset
data.to_csv("cleaned_car.csv", index=False)

In [ ]:
print(data.head())

  registration_date    year  price_in_euro  power_kw  power_ps  \
0        2023-05-01  2023.0        38490.0  0.069154  0.233622   
1        2018-09-01  2018.0        11555.0  0.023881  0.081582   
2        2021-05-01  2021.0        48886.0  0.061692  0.208900   
3        2018-07-01  2018.0        24490.0  0.056219  0.190358   
4        2023-02-01  2023.0        34990.0  0.054229  0.184178   

   fuel_consumption_l_100km  fuel_consumption_g_km  mileage_in_km  \
0                  0.002669               0.061295       0.000026   
1                  0.002274               0.052397       0.007311   
2                  0.002867               0.000000       0.001118   
3                  0.003707               0.086505       0.015000   
4                  0.002916               0.074147       0.001974   

   registration_month  car_age  ...  fuel_type_diesel hybrid  \
0                   5        2  ...                        0   
1                   9        7  ...                        0

In [117]:
data.isnull().sum()

,0
registration_date,0
year,0
price_in_euro,0
power_kw,0
power_ps,0
...,...
fuel_type_lpg,0
fuel_type_manual,0
fuel_type_other,0
fuel_type_petrol,0


In [ ]:
# #histogram to understand the distribution
# import warnings
# warnings.filterwarnings("ignore")
# for i in data.select_dtypes(include="number").columns:
#     plt.figure(figsize=(10,6))
#     sns.histplot(data=data,x=i,kde=True)
#     plt.show()

In [ ]:
# Count the number of occurrences for each body type
plt.figure(figsize=(10,6))
sns.countplot(data=data, x='brand')  # Replace 'body_type' with the correct column name
plt.title('Count of brand')
plt.xlabel('Brand')
plt.ylabel('Count')
plt.xticks(rotation=45)  # Rotate x-axis labels if needed
plt.show()